# Congressional Voting Networks Analysis

This notebook demonstrates how to use the Congressional Voting Networks package to analyze US Congress voting patterns using network analysis techniques.

## Contents
1. Data Loading and Preprocessing
2. Network Construction
3. Centrality Analysis
4. Community Detection
5. Polarization Metrics
6. Visualization
7. Temporal Analysis

In [ ]:
# Import required packages
import sys
sys.path.insert(0, '..')

import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

# Configure logging
logging.basicConfig(level=logging.INFO)

# Import our modules
from src.data_acquisition import VoteviewDataLoader
from src.preprocessing import VoteDataPreprocessor
from src.network_builder import CongressionalNetworkBuilder
from src.analysis import NetworkAnalyzer
from src.visualization import NetworkVisualizer

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

## 1. Data Loading and Preprocessing

First, we'll load the voting data from Voteview.com. This may take a moment if downloading for the first time.

In [ ]:
# Initialize data loader
loader = VoteviewDataLoader('../data/raw')

# Download data if needed
loader.download_all_data()

# Load data for recent Congresses (117th and 118th)
congress_range = (117, 118)

members = loader.load_members(congress_range=congress_range)
rollcalls = loader.load_rollcalls(congress_range=congress_range)
votes = loader.load_votes(congress_range=congress_range)

print(f"Loaded {len(members)} member records")
print(f"Loaded {len(rollcalls)} rollcall records")
print(f"Loaded {len(votes)} vote records")

In [ ]:
# Preview the data
members.head()

In [ ]:
# Preprocess the data
preprocessor = VoteDataPreprocessor(members, rollcalls, votes)
preprocessor.preprocess_all()

# Show party distribution
preprocessor.members.groupby(['congress', 'party_name']).size().unstack().fillna(0)

## 2. Network Construction

Now we'll build a co-voting network for a specific Congress.

In [ ]:
# Create vote matrix for 118th Congress
matrix, leg_ids, roll_ids = preprocessor.create_vote_matrix(congress=118)

print(f"Vote matrix shape: {matrix.shape}")
print(f"Number of legislators: {len(leg_ids)}")
print(f"Number of rollcalls: {len(roll_ids)}")

In [ ]:
# Get legislator info
leg_info = preprocessor.get_legislator_info(leg_ids)
leg_info.head()

In [ ]:
# Build the network
builder = CongressionalNetworkBuilder(matrix, leg_ids, leg_info)

# Compute similarity matrix
similarity = builder.compute_similarity_matrix(method='cosine')

# Build network with threshold
G = builder.build_similarity_network(
    similarity_threshold=0.6,
    method='cosine'
)

print(f"Network has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")

## 3. Centrality Analysis

Let's identify the most influential legislators using various centrality measures.

In [ ]:
# Initialize analyzer
analyzer = NetworkAnalyzer(G)

# Compute all centralities
centralities = analyzer.compute_all_centralities()
centralities.head(10)

In [ ]:
# Top legislators by eigenvector centrality
top_legislators = analyzer.get_top_legislators(centrality_type='eigenvector', top_n=10)
print("Top 10 Legislators by Eigenvector Centrality:")
top_legislators[['bioname', 'party_group', 'state_abbrev', 'centrality']]

In [ ]:
# Top legislators by party
top_by_party = analyzer.get_top_legislators(centrality_type='betweenness', top_n=5, by_party=True)
print("Top 5 Legislators per Party by Betweenness Centrality:")
top_by_party[['bioname', 'party_group', 'state_abbrev', 'centrality']]

## 4. Community Detection

We'll use the Louvain algorithm to detect communities in the network.

In [ ]:
# Detect communities
communities = analyzer.detect_communities_louvain()
n_communities = len(set(communities.values()))
print(f"Detected {n_communities} communities")

# Get community summary
community_summary = analyzer.get_community_summary()
community_summary

In [ ]:
# Compare communities to parties
comparison = analyzer.compare_communities_to_parties()
print(f"Normalized Mutual Information with party labels: {comparison['nmi']:.4f}")
print(f"(1.0 = perfect alignment, 0.0 = no alignment)")

## 5. Polarization Metrics

Let's examine polarization in Congress.

In [ ]:
# Compute polarization metrics
polarization = analyzer.compute_polarization_score()
assortativity = analyzer.compute_party_assortativity()
edge_stats = analyzer.compute_cross_party_edge_ratio()
cohesion = analyzer.compute_party_cohesion()

print(f"Polarization Score: {polarization:.4f}")
print(f"Party Assortativity: {assortativity:.4f}")
print(f"Cross-Party Edge Ratio: {edge_stats['cross_party_ratio']:.4f}")
print(f"\nParty Cohesion:")
for party, score in cohesion.items():
    print(f"  {party}: {score:.4f}")

In [ ]:
# Generate full report
print(analyzer.generate_report())

## 6. Visualization

Let's visualize the network and analysis results.

In [ ]:
# Initialize visualizer
viz = NetworkVisualizer(G, figsize=(14, 12))

# Plot network colored by party
fig = viz.plot_network(
    layout='spring',
    color_by='party',
    title='118th Congress Voting Network (Colored by Party)',
    edge_alpha=0.1
)
plt.show()

In [ ]:
# Plot with party-split layout
fig = viz.plot_network(
    layout='party_split',
    color_by='party',
    title='118th Congress - Party Split Layout',
    edge_alpha=0.1
)
plt.show()

In [ ]:
# Plot community structure
fig = viz.plot_community_structure(
    communities,
    title='Detected Community Structure'
)
plt.show()

In [ ]:
# Plot DW-NOMINATE positions
fig = viz.plot_nominate_scatter(
    title='DW-NOMINATE Ideological Positions (118th Congress)'
)
plt.show()

In [ ]:
# Plot centrality distribution
fig = viz.plot_centrality_distribution(
    centralities,
    centrality_type='eigenvector',
    title='Eigenvector Centrality Distribution by Party'
)
plt.show()

In [ ]:
# Plot similarity heatmap
party_labels = [leg_info.loc[lid, 'party_group'] if lid in leg_info.index else 'Unknown' 
                for lid in leg_ids]

fig = viz.plot_similarity_heatmap(
    similarity,
    party_labels=party_labels,
    title='Legislator Voting Similarity (sorted by party)'
)
plt.show()

## 7. Temporal Analysis

Let's analyze how polarization has changed over recent Congresses.

In [ ]:
from src.network_builder import build_temporal_networks
from src.analysis import analyze_temporal_networks

# Build networks for multiple Congresses (this may take a few minutes)
congress_range = (110, 118)

# Load more data if needed
full_members = loader.load_members(congress_range=congress_range)
full_rollcalls = loader.load_rollcalls(congress_range=congress_range)
full_votes = loader.load_votes(congress_range=congress_range)

full_preprocessor = VoteDataPreprocessor(full_members, full_rollcalls, full_votes)
full_preprocessor.preprocess_all()

# Build temporal networks
networks = build_temporal_networks(
    full_preprocessor,
    congress_range=congress_range,
    similarity_threshold=0.6
)

print(f"Built {len(networks)} networks")

In [ ]:
# Analyze temporal networks
temporal_df = analyze_temporal_networks(networks)
temporal_df

In [ ]:
# Plot polarization trend
fig = viz.plot_polarization_over_time(
    temporal_df,
    metrics=['polarization', 'assortativity'],
    title=f'Congressional Polarization Trend ({congress_range[0]}-{congress_range[1]})'
)
plt.show()

In [ ]:
# Plot party cohesion trend
fig = viz.plot_party_cohesion_over_time(
    temporal_df,
    title=f'Party Cohesion Trend ({congress_range[0]}-{congress_range[1]})'
)
plt.show()

## Summary

This notebook demonstrated the full workflow for analyzing Congressional voting networks:

1. **Data Loading**: Downloaded and loaded voting data from Voteview.com
2. **Preprocessing**: Cleaned and transformed raw data into analyzable formats
3. **Network Construction**: Built co-voting similarity networks
4. **Centrality Analysis**: Identified influential legislators
5. **Community Detection**: Found voting coalitions using Louvain algorithm
6. **Polarization Metrics**: Quantified partisan division
7. **Visualization**: Created static visualizations of the network
8. **Temporal Analysis**: Tracked changes in polarization over time

For more detailed analysis, see the `main.py` script for command-line usage.